# Pipette Intro
This example notebook will show you how to do simple liquid handling with an OT-2 tool. It assumes you know a little bit about the [laboratory automation deck](../labware/0_AutomationDeckIntro.ipynb) and [installing labware](../labware/1_LabwareIntro.ipynb).

### Before Starting:
- Clear any existing items off the bed of your machine!
- A lab automation deck and pipette should be installed on your machine

In [ ]:
### Import modules and connect to the machine
from science_jubilee.Machine import Machine
from science_jubilee.decks import Deck
from science_jubilee.tools import Pipette

In [ ]:
m = Machine(address='10.19.103.41') # connect to the machine

In [ ]:
deck = m.load_deck('lab_automation_deck') # load in your deck calibration

### Labware Preparation
Now we need to load all labware both *physically* and *digitally* . 

* Digitally: we'l load the `labware` we want to use and indicate which `slot` each is assigned to
* Physically: install the labware onto the deck of your Jubilee

For this example, we'll use the following labware:

* A **Tiprack** that is suitable for the pipette you are using to transfer your liquids. We'll use an OpenTrons tiprack. 
* A **Sample Labware** that will be used to collect samples. We'll use a 96 well plate.
* A **Stock Labware** that will host your stock liquids. We'll use a 6-well plate.
* A **trash** to collect used tips. We'll use a petri dish.

If you'd like, you can fill your stock labware (the 24 well plate) wells A1 and A2 with water. You're welcome to add water color/etc as well, or just leave them empty to do a dry run.

In [ ]:
# -------------- Labware ------------------
tiprack = m.load_labware('opentrons_96_tiprack_300ul', 0)       # OpenTrons tip rack in slot 0
trash = m.load_labware('generic_petri_dish_100ml', 1)           # Petri dish in slot 1
samples = m.load_labware('fisherbrand_96_wellplate_360ul', 2)   # 96 well plate in slot 2
stocks = m.load_labware('greiner_24_wellplate_3300ul', 3)       # 24 well plate in slot 3


### Tool Setup

To use your tool, you'll need to set a `tool_index` and a `tool_name`. These should be the same as the ones defined in your machine's `config.g` file that appear in the Duet Web Control panel.

You will also import a configuration file for your pipette. If you're using a P300 pipette, use the `P300_config`; if you have a P1000 pipette, use `P1000_config`!

In [ ]:
pipette_index = 3           # change this number to match the tool number for the pipette on your machine!
pipette_config = "P300_config"  # change this to P300_config for a P300 pipette, P1000_config for a P1000 pipette
pipette_name = "Pipette"        # No need to change this


In [ ]:
pipette = Pipette.Pipette.from_config(pipette_index, pipette_name, pipette_config) 
m.load_tool(pipette)

Even though you will be able to use te pipette after running the above cell, you can associate its `tiprack` to the tool, as well as define a `trash` location. This will help keep the code readable


In [ ]:
pipette.add_tiprack(tiprack)
pipette.trash = trash[0]

We can now pickup our tool and start playing around with it!

In [ ]:
m.pickup_tool(pipette)

## Moving Liquids
Now, that our tool is active, we can choose start moving liquids around. In the following cells, we will:
* pickup a tip
* aspirate a certain volume ( in $\mu$ L) from a source reservoir/well
* dispense it into a destination well
* return the tip ( if we want to reuse it) *OR* drop the tip in the trash

In [ ]:
# by default, we'll pickup the first tip in the tiprack
# make sure your tiprack is loaded up with tips!
pipette.pickup_tip() 

In [ ]:
# Now we'll pick up some liquid from our stock plate
pipette.aspirate(250, stocks['A1'].bottom(3)) # (volume in microliters, well to aspirate from)

In [ ]:
# Nowd dispense into our samples plate
pipette.dispense(250, samples['A1'].top(-1)) # (volume in microliters, well to dispense to)

Once we are done, we can drop our tip in our 'trash'

In [ ]:
pipette.drop_tip()

## Tranfser Function
Above, we used `pickup_tip`, `aspirate`, `dispense`, and `drop_tip`to manually  take care of our liquid handling. We can instead use `transfer()` which will take care of both at the same time.

In [ ]:
pipette.transfer(
            vol = 250,                                    # volume to transer in microliters
            source_well = stocks['A1'].bottom(5),         # source well to transer from
            destination_well = samples['A2'].top(-1),     # destination well to transfer to
            blowout = True,                               # blowout after dispensing to make sure there's not dripping
            mix_after = (200, 3)                          # mix after dispensing (volume in microliters, number of times to mix)
            )

You can also provide a list of volumes and source_wells to make things a bit easier. The index of volumes and source_wells are 1:1. 

In [ ]:
volumes = [50, 100, 50]                                                              # transfer 50 microliters, then 100, then 50 
stocks = [stocks['A1'].bottom(5), stocks['A2'].bottom(5), stocks['A3'].bottom(5)]    # transfer from stock wells A1, then A2, then A3
destination = samples['A3'].top(-1)                                                  # move all samples into sample well A3

pipette.transfer(
            vol = volumes,
            source_well = stocks,
            destination_well = destination,
            blowout = True, 
            new_tip = 'once',
            mix_after = (200, 3)
)

In the above transfer function, we used `new_tip = 'once'` to use the same tip for all the transfers. We could alternatively use `new_tip = 'always'` to get a new pipette tip for every transfer, or `new_tip = 'never'` if we already have a tip attached

## Next Steps
If you want more pipette examples, you can look at the [serial dilution example notebook](./1_SerialDilution.ipynb)